In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

class ApparelModel(nn.Module):
    def __init__(self):
        super(ApparelModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 512, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            nn.Conv2d(512, 256, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            nn.Conv2d(256, 128, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            nn.Conv2d(128, 128, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 14 * 14, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            nn.Linear(64, 37),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Load the model
model = ApparelModel()
model.load_state_dict(torch.load('recognition-models/model_checkpoint_v2.pth', map_location=torch.device('cpu')))
# model.load_state_dict(torch.load('model_checkpoint.pth', map_location=torch.device('cpu')))
model.eval()  # Set model to evaluation mode


ApparelModel(
  (conv_layers): Sequential(
    (0): Conv2d(3, 512, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
    (5): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.2, inplace=False)
    (10): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout(p=0.2, inplace=False)
    (15): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (16): ReLU()
    (

In [4]:
# Define the image preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to match the expected input size (assuming it's 224x224)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess the image
def load_image(image_path):
    image = Image.open(image_path)
    image = preprocess(image)
    image = image.unsqueeze(0)
    return image

# Path to your image
image_path = 'test-images/IMG_7247.jpeg'
image = load_image(image_path)


In [5]:
# Make predictions
with torch.no_grad():
    output = model(image)

# Get the predicted class
_, predicted_class = torch.max(output, 1)

# class_names = [
#     "black dress/",
#     "black pants/",
#     "black shirt/",
#     "black shoes/",
#     "black shorts/",
#     "black suit/",
#     "blue dress/",
#     "blue pants/",
#     "blue shirt/",
#     "blue shoes/",
#     "blue shorts",
#     "brown hoodie",
#     "brown pants/",
#     "brown shoes/",
#     "green pants/",
#     "green shirt/",
#     "green shoes/",
#     "green shorts/",
#     "green suit/",
#     "pink hoodie/",
#     "pink pants/",
#     "pink skirt/",
#     "red dress/",
#     "red hoodie/",
#     "red pants/",
#     "red shirt/",
#     "red shoes/",
#     "silver shoes",
#     "silver skirt/",
#     "white dress/",
#     "white pants/",
#     "white shirt/",
#     "white shoes/",
#     "white shorts/",
#     "white suit/",
#     "yellow dress/",
#     "yellow shoes/",
#     "yellow shirt/"
# ]

class_names = [
    "dress/",
    "hoodie/",
    "pants/",
    "shirt/",
    "shoes/",
    "shorts/",
    "skirt/",
    "suit/",
]

predicted_class_name = class_names[predicted_class.item()]

print(predicted_class_name)


dress/
